# Monto carlo simulations

In [ ]:
# %load imports.py
%load_ext autoreload
%autoreload 2
%reload_kedro
%config Completer.use_jedi = False  ## (To fix autocomplete)
import pandas as pd
from vessel_manoeuvring_models.models.vmm import ModelSimulator
import matplotlib.pyplot as plt
from vessel_manoeuvring_models.visualization.plot import track_plots, plot, captive_plot
import kedro
import numpy as np
import os.path
import anyconfig

import matplotlib
matplotlib.rcParams["figure.figsize"] = (15,4)
from vessel_manoeuvring_models.symbols import *

# Read configs:
conf_path = os.path.join("../conf/base/")
runs_globals_path = os.path.join(
    conf_path,
    "runs_globals.yml",
)

runs_globals = anyconfig.load(runs_globals_path)
model_test_ids = runs_globals["model_test_ids"]

join_globals_path = os.path.join(
    conf_path,
    "join_globals.yml",
)

joins = runs_globals["joins"]
join_runs_dict = anyconfig.load(join_globals_path)

globals_path = os.path.join(
    conf_path,
    "globals.yml",
)
global_variables = anyconfig.load(globals_path)



vmms = global_variables["vmms"]
only_joined = global_variables[
    "only_joined"
]  # (regress/predict with only models from joined runs)S

In [ ]:
from wPCC_pipeline.pipelines.motion_regression.nodes import fit_motions
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedKFold

In [ ]:
join = "joined"
#vmm_name = "vmm_abkowitz"
vmm_name = "vmm_martins_simple"


data = catalog.load(f"{ join }.data_ek_smooth")

model = catalog.load(f"{ vmm_name }.motion_regression.{ join }.model")
ek = catalog.load(f"{ vmm_name }.ek")
added_masses = catalog.load("added_masses")
ship_data = catalog.load("ship_data")
vmm = catalog.load(f"{vmm_name}")
exclude_parameters = catalog.load("params:motion_regression.exclude_parameters")

In [ ]:
for key in ['u1d','v1d','r1d']:
    
    fig,ax=plt.subplots()
    fig.set_size_inches(10,5)
    data.plot(y=key, ax=ax)
    ax.set_title(key)
    

In [ ]:
np.random.seed(42)
regression, parameters = fit_motions(data=data, added_masses=added_masses, ship_data=ship_data, vmm=vmm, exclude_parameters=exclude_parameters)

In [ ]:
regression.model_X.summary()

In [ ]:
regression.model_X.bse

In [ ]:
regression.std.head()

In [ ]:
regression.parameters['regressed'].head()

In [ ]:
regression.parameters.plot.bar(y='regressed', yerr=regression.std)

In [ ]:
parameters = pd.DataFrame()
parameters['mean'] = regression.parameters['regressed']
parameters['std'] = regression.std

In [ ]:
def plot_parameters(parameters, quantile_cuts = [0.6,0.95]):

    cuts = [0] + quantile_cuts + [1]
    cuts = np.flipud(cuts)
    
    N = len(cuts)-1
    fig,axes=plt.subplots(nrows=N)
    fig.set_size_inches(12,12)
    
    for i,ax in zip(range(N),axes):
        
        cut_start = cuts[i+1]
        cut_stop = cuts[i]
            
        mask = ((parameters['mean'].abs() > parameters['mean'].abs().quantile(cut_start)) & 
               (parameters['mean'].abs() <= parameters['mean'].abs().quantile(cut_stop)))
        
        parameters.loc[mask].plot.bar(y='mean', yerr=parameters['std'], ax=ax)
        
    plt.tight_layout()
    return fig

In [ ]:
plot_parameters(parameters);

In [ ]:
from scipy.stats import norm, multivariate_normal
from wPCC_pipeline.pipelines.prediction.nodes import simulate_euler

In [ ]:
means = regression.parameters['regressed']
stds = regression.std

cov = np.diag(stds.values**2)
cov = regression.covs.values

rv = multivariate_normal(mean=means, cov=cov, allow_singular=True)
np.random.seed(42)
N_=20
df_parameter_variation = pd.DataFrame(data=rv.rvs(N_), columns=means.index)

In [ ]:
np.diagonal(cov)

In [ ]:
np.diagonal(regression.covs**4)

In [ ]:
id = 22772
df_smooth = catalog.load(f"{ id }.data_ek_smooth")

dataframes = {
}

In [ ]:
for index, parameters_ in df_parameter_variation.iterrows():
    model_ = model.copy()
    model_.parameters.update(parameters_)
    df_ = simulate_euler(data=df_smooth, model=model_,ek=ek, solver='Radau') 
    dataframes[index] = df_


dataframes['manoeuvring model'] = simulate_euler(data=df_smooth, model=model,ek=ek, solver='Radau') 
dataframes['model test'] = df_smooth

In [ ]:
styles={'model test':{'style':'k-'},
       'manoeuvring model':{'style':'y--'},
       }


fig,ax=plt.subplots()
fig.set_size_inches(10,10)
track_plots(dataframes, lpp=ship_data['L'], beam=ship_data['B'], N=7, plot_boats=False, ax=ax, styles=styles)
ax.set_xlim(-10,10)

plot(dataframes, styles=styles);

In [ ]:
regression.model_X.cov_HC0.shape

In [ ]:
from vessel_manoeuvring_models.substitute_dynamic_symbols import run
from vessel_manoeuvring_models.parameters import df_parameters
p = df_parameters['symbol']

In [ ]:
from vessel_manoeuvring_models.parameters import df_parameters

In [ ]:
parameters.sort_index(inplace=True)
parameters['symbol'] = df_parameters.loc[parameters.index]['symbol']
parameters['name'] = parameters.index
parameters['dof'] = parameters['name'].apply(lambda x:x[0])

In [ ]:
parameters